In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

from load_eval import *
import torchmetrics
from tqdm import tqdm
import clip

seed_everything(hparams['seed'])

In [2]:
bs = hparams['batch_size']
dataset = ImageNet(hparams['data_dir'], split='val', transform=tfms)
# dataset = CUBDataset(hparams['data_dir'], train=False, transform=tfms)
# dataset = torchvision.datasets.OxfordIIITPet(root=hparams['data_dir'], transform=tfms, split='test')
# dataset = test_set # EuroSAT
# dataset = torchvision.datasets.Food101(root=hparams['data_dir'], transform=tfms, split='test')

dataloader = DataLoader(dataset, bs, shuffle=True, num_workers=16, pin_memory=True)

In [ ]:
print("Loading model...")

device = torch.device(hparams['device'])
# load model
model, preprocess = clip.load(hparams['model_size'], device=device, jit=False)
checkpoint = torch.load("/path/to/your/model.pt") 
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
model.requires_grad_(False)

In [ ]:
print("Encoding descriptions...")

description_encodings = compute_description_encodings(model)
label_encodings = compute_label_encodings(model)

In [ ]:
print("Evaluating...")
lang_accuracy_metric = torchmetrics.Accuracy().to(device)
lang_accuracy_metric_top5 = torchmetrics.Accuracy(top_k=5).to(device)

clip_accuracy_metric = torchmetrics.Accuracy().to(device)
clip_accuracy_metric_top5 = torchmetrics.Accuracy(top_k=5).to(device)

In [ ]:
for batch_number, batch in enumerate(tqdm(dataloader)):
    images, labels = batch
    
    images = images.to(device)
    labels = labels.to(device)
    
    image_encodings = model.encode_image(images)
    image_encodings = F.normalize(image_encodings)
    
    image_labels_similarity = image_encodings @ label_encodings.T
    clip_predictions = image_labels_similarity.argmax(dim=1)
    
    
    clip_acc = clip_accuracy_metric(image_labels_similarity, labels)
    clip_acc_top5 = clip_accuracy_metric_top5(image_labels_similarity, labels)
    
    
    image_description_similarity = [None]*n_classes
    image_description_similarity_cumulative = [None]*n_classes
    
    for i, (k, v) in enumerate(description_encodings.items()): # You can also vectorize this; it wasn't much faster for me
        
        
        dot_product_matrix = image_encodings @ v.T
        
        image_description_similarity[i] = dot_product_matrix
        image_description_similarity_cumulative[i] = aggregate_similarity(image_description_similarity[i])
        
        
    # create tensor of similarity means
    cumulative_tensor = torch.stack(image_description_similarity_cumulative,dim=1)
        
    
    descr_predictions = cumulative_tensor.argmax(dim=1)
    
    
    lang_acc = lang_accuracy_metric(cumulative_tensor.softmax(dim=-1), labels)
    lang_acc_top5 = lang_accuracy_metric_top5(cumulative_tensor.softmax(dim=-1), labels)
    
    

print("\n")

accuracy_logs = {}
accuracy_logs["Total Concept-based Top-1 Accuracy: "] = 100*lang_accuracy_metric.compute().item()
accuracy_logs["Total Concept-based Top-5 Accuracy: "] = 100*lang_accuracy_metric_top5.compute().item()

accuracy_logs["Total CLIP-Standard Top-1 Accuracy: "] = 100*clip_accuracy_metric.compute().item()
accuracy_logs["Total CLIP-Standard Top-5 Accuracy: "] = 100*clip_accuracy_metric_top5.compute().item()

# print the dictionary
print("\n")
for key, value in accuracy_logs.items():
    print(key, value)